# Generalized Cellular Automata Test
The goal of this project is to experiment with a new paradigm for generalized intelligence, inspired heavily by (this)[https://www.youtube.com/watch?v=p-OYPRhqRCg] talk between Joscha Bach and Michael Levin. The ideas I'm drawing upon are primarily Bachs, however I take heavy inspiration by Levins work on collective intelligence in all forms of cells as well, seeking originally to make one based mostly on the idea of bioelectric signaling, and slowly shifting more towards Joscha Bachs idea on a search for selector_functions to find appropraite neighbors in higher dimensions to process information over long distances.

In [44]:
import os
import imageio
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import csv

# Create a unique directory name based on the current date and time
current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
experiment_dir = f'Experiments/Experiment_{current_time}'
os.makedirs(experiment_dir, exist_ok=True)

def save_grid_data(grid, step, save_dir):
    with open(f'{save_dir}/grid_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        for row in grid:
            writer.writerow([step] + [cell.state for cell in row])



def save_grid_image(grid, step, save_dir):
    image = np.array([[cell.state for cell in row] for row in grid])
    plt.imshow(image, cmap='gray')
    plt.title(f'Step {step}')
    plt.savefig(f'{save_dir}/step_{step}.png')
    plt.close()

def create_images_from_csv(save_dir):
    with open(f'{save_dir}/grid_data.csv', 'r') as file:
        reader = csv.reader(file)
        current_step = -1
        grid = []

        for row in reader:
            step = int(row[0])
            if step != current_step:
                if current_step != -1:
                    # Save the previous step's image
                    grid_array = np.array(grid)
                    plt.imshow(grid_array, cmap='gray')
                    plt.title(f'Step {current_step}')
                    plt.savefig(f'{save_dir}/step_{current_step}.png')
                    plt.close()

                current_step = step
                grid = []

            grid.append([float(cell) for cell in row[1:]])

        # Save the last step's image
        if grid:
            grid_array = np.array(grid)
            plt.imshow(grid_array, cmap='gray')
            plt.title(f'Step {current_step}')
            plt.savefig(f'{save_dir}/step_{current_step}.png')
            plt.close()

 
def create_gif(image_folder, gif_name):
    images = []

    def sort_key(file_name):
        numbers = re.findall(r'\d+', file_name)
        return int(numbers[0]) if numbers else 0

    file_names = sorted(os.listdir(image_folder), key=sort_key)

    for file_name in file_names:
        if file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))

    imageio.mimsave(os.path.join(image_folder, gif_name), images, fps=10)

    for file_name in file_names:
            if file_name.endswith('.png'):
                file_path = os.path.join(image_folder, file_name)
                os.remove(file_path)

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.state = 0.0  # Use a floating-point number for continuous states
        self.memory = []
        self.selector = []  # Initialize with basic neighbors
        self.lastreward = []
        # self.lookback = random.randramge(4, 9)

    def update_selector(self):
        # Placeholder logic: Randomly selects neighbors as a simple example
        # In practice, this should be more sophisticated based on the cell's history and global states
        self.selector = np.random.choice([True, False], size=(grid_size[0], grid_size[1]))

    def update_state(self, new_state, memory):
        self.state = new_state
        self.memory.append(new_state)

    def get_recent_memory(self):
        return self.memory[-10:]  # Return the last 10 states

# Function to get neighboring cells based on selector
def get_neighbors(cell):
    neighbors = []
    for di in range(-1, 2):
        for dj in range(-1, 2):
            if di == 0 and dj == 0:
                continue  # Skip the cell itself
            ni, nj = cell.x + di, cell.y + dj
            if 0 <= ni < grid_size[0] and 0 <= nj < grid_size[1] and cell.selector[ni][nj]:
                neighbors.append(ca_grid[ni][nj])
    return neighbors

# Define the cell update logic for continuous values
def update_cell_state(cell, neighbors):
    # Example rule for continuous values
    active_neighbors = sum(neighbor.state for neighbor in neighbors)
    new_state = active_neighbors / len(neighbors) if neighbors else 0
    memory = [new_state, [neighbors]] if neighbors else [new_state, []]
    cell.update_state(new_state, memory)

# Learning mechanism
def apply_learning(cell):
    # Example learning rule for continuous values
    if sum(cell.get_recent_memory()) / len(cell.get_recent_memory()) < 0.3:  # Low average activation
        # cell.state += 0.1  # Increase sensitivity
        cell.update_selector()
        
# Initialize the CA grid with Cell objects
grid_size = (100, 100)
ca_grid = np.empty(grid_size, dtype=object)
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        ca_grid[i, j] = Cell(i, j)
        ca_grid[i, j].update_selector()

# Main simulation loop
simulation_steps = 100
for step in range(simulation_steps):
    # Update each cell based on rules
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            cell = ca_grid[i, j]
            neighbors = get_neighbors(cell)
            update_cell_state(cell, neighbors)
            apply_learning(cell)
    print(step)
    # Visualize the grid state
    # plt.figure()  # Create a new figure
    # plt.imshow([[cell.state for cell in row] for row in ca_grid], cmap='norm')
    # plt.title(f'Step {step}')
    # plt.show()
    # plt.close()  # Close the figure

    
    # Save as image for visualization
    # save_grid_image(ca_grid, step, experiment_dir)
    save_grid_data(ca_grid, step, experiment_dir)

print("Done running, processing image")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Done running, processing image


In [45]:
create_images_from_csv(experiment_dir)
create_gif(experiment_dir, 'ca_simulation.gif')

C:\Users\CYBER\AppData\Local\Temp\ipykernel_13324\3487693905.py:72: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


# Random ideas
* Now that we have the cellular automata engine, it's time to make it think. The basic idea is to have it's selector function decide which neighbors it looks for, probably up to the maximum of 8 it coulde have around it spatially. We can intialize them entirely randomly, or simply start with it's base neighbors.
* To get useful inputs and outputs, we need to have certain neurons represent inputs, and others represent outputs. For instance, we could encode some information in the continuous value of input neurons, with more neurons used for more complex information, and have the available actions mapped to certain outputs (ideally these outputs would be more arbitary and less hard-coded somehow in the future).
* The learning mechanism will be reward driven using an economic distribution sort of appraoch similar to the suggestions given by Joscha Bach in the Generalist AI talk between Joscha Bach and Michael Levin. We will attempt to track how much each neuron contributes to the final reward.
* Input neurons would be rewarded more for keeping the input information to encourage the system to keep the information.
* A basic training test we can imagine is a simple addition algorithm. For instance, we can create a simple 4 bit adder. To do so, we will have 2 sets of 4 cell inputs, and one set of a 4 bit output with a carry output.
* If a cell is negatively contributing to the loss function (or whatever metric we use to evaluate the result), cells will weaken their connection to it and eventually seek new neighbors, and if they're positively contributing cells will strengthen their connection. This can draw on the cells memory in some fashion as well.
* If a cell is itself negatively contributing to the loss function, it will randomly seek new neighbors in a hope to eventually contribute.
* Unlike a traditional neural net which would be trained and then ran in inference, the training of this automata would be continuously ongoing. I'll need to add ways to interact with it live to help test how resistant it is to perturbations over time and learning entirely new tasks within the network, for isntance starting with a 2 bit adder, upping it to 4, upping that to 8, etc.
* Since information would take time to propagate through the network and calculate (unless there was somehow direct connections to the task, but in practice the tasks should be too complex for such simple connections), the method by which rewards are propagated through the network should probably be analyzed every x steps or something of the sort. The memory could in theory play some sort of part in this, and in general it seems to line up with my recent thoughts on the importance of brain-wave like patterns in intelligence in general. This could in practice mean we can give a new input and read the output every x steps, say 10, making the network have an effective processing time of x steps * time per step.